<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/lindevanbeers/Lab-2/linde-report-lab-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Linde van Beers**

# **Abstract**



```
# This is formatted as code
```

Short summary of the lab report. State the objectives, methods used, main results and conlusions. 

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. 

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method. 

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/7500)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$



# **Methods**

Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.  

### Sparse matrix-vector product

In [0]:
def sparse_MVP(x,A_val,col_idx, row_ptr):
  # Assume that row_ptr has an extra element set to NNZ
  n = len(row_ptr)-1
  b = np.zeros(n)
  for i in range(n):
    for j in range(row_ptr[i], row_ptr[i+1]):
      b[i] += A_val[j]*x[col_idx[j]]
  return b

### QR factorization

for k=0:n-1 do

  x[:] = A[k:n-2,k]

  v_k[:] = x[:]

  v_k[0] = v_k[0] - sign(x[0])*norm(x) 
  
  v_k[:] = v_k[:]/norm(v_k)

  for m=k:n-1 do

A[k:n-1,m] = A[k:n-1,m] - 2*v_k[:]*dot(v_k, A[k:n-1,m])

end for 

endfor

return v_0,..,v_n-2

In [0]:
def qr_householder(A):
  n = A.shape[1]
  for k in range(n-1):
    x = A[k:n,k]
    v_k = x.copy()
    v_k[0] = v_k[0]-np.sign(x[0])*np.linalg.norm(x)
    v_k = v_k/np.linalg.norm(v_k)
    for m in range(k,n):
      A[k:n,m] = A[k:n,m]-(2*v_k*(np.dot(v_k.T,A[k:n,m])))  
    Q_k = np.eye(n,n)
    Q_k[k:n,k:n]= Q_k[k:n,k:n]-(2*(np.matmul(v_k,v_k.T))/np.dot(v_k.T,v_k))
    if k==0:
      Q = Q_k
    else:
      Q = np.matmul(Q.T,Q_k.T)
  R=A
  return Q,R

### Direct solve

In [280]:
def direct_solve(A,b):
  n = A.shape[1]
  m = A.shape[0]
  x = np.zeros([n-1,1])

  
  sys = np.append(A,b, axis=1)
  if sys[0,0]==0:
    for i in range(m):
      if sys[i,0] != 0:
        sys[0,:] = sys[0,:] + sys[i,:]/sys[i,0]
        break
  for i in range(m-1):
    for j in range(i+1,m):
      c = sys[j,i]/sys[i,i]
      sys[j,:] = sys[j,:] - c*sys[i,:]
  print(sys)
  b = np.matrix(sys[:,n])
  A = np.matrix(sys[:,0:n])
  print(A)
  print(b)
  
  

A = np.matrix([[0.,4.,1.],[1.,3.,-2.],[-1.,-2.,2.]])
b = np.matrix([[1.,2.,3.]]).T
direct_solve(A,b)

A = np.matrix([[1.,3.,-2.],[0.,4.,1.],[-1.,-2.,2.]])
b = np.matrix([[2.,1.,3.]]).T
direct_solve(A,b)

[[ 1.    7.   -1.    3.  ]
 [ 0.   -4.   -1.   -1.  ]
 [ 0.    0.   -0.25  4.75]]
[[ 1.    7.   -1.  ]
 [ 0.   -4.   -1.  ]
 [ 0.    0.   -0.25]]
[[ 3.  ]
 [-1.  ]
 [ 4.75]]
[[ 1.    3.   -2.    2.  ]
 [ 0.    4.    1.    1.  ]
 [ 0.    0.   -0.25  4.75]]
[[ 1.    3.   -2.  ]
 [ 0.    4.    1.  ]
 [ 0.    0.   -0.25]]
[[2.  ]
 [1.  ]
 [4.75]]


# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data. 

### Sparse matrix-vector product

Test

In [108]:
A = np.matrix([[0,1,0],[2,0,3],[0,0,4]])
x = [1,2,3]
print("A:",A)
print("x:",x)

# Create a sparse matrix object 
val = np.array([1,2,3,4])
col_idx = np.array([1,0,2,2])
row_ptr = np.array([0,1,3,4])

# Print sparse matrix object attributes
print('Sparse matrix values: \n',val)
print('Sparse matrix column indices: \n',col_idx)
print('Sparse matrix row pointer: \n',row_ptr)

# Calculate b using the sparse matrix
b = sparse_MVP(x, val, col_idx, row_ptr)
# Calculate b using dense matrix 
bAns = A*np.matrix(x).T

print("answer we are looking for:",bAns.T)
print("answer by function:",b)

A: [[0 1 0]
 [2 0 3]
 [0 0 4]]
x: [1, 2, 3]
Sparse matrix values: 
 [1 2 3 4]
Sparse matrix column indices: 
 [1 0 2 2]
Sparse matrix row pointer: 
 [0 1 3 4]
answer we are looking for: [[ 2 11 12]]
answer by function: [ 2. 11. 12.]


### QR Factorization

In [273]:
A = np.matrix([[12,-51,4],[6,167,-68],[-4,24,-41]])
print("A",A)
Q,R = qr_householder(A)
print("Q",Q)
print("R",R)
print("QR", np.matmul(Q,R))
print("\n")
A = np.matrix([[2.,-2.,18],[2.,1.,0.],[1.,2.,0.]])
print("A",A)
Q,R = qr_householder(A)
print("Q",Q)
print("R",R)
print("QR", np.matmul(Q,R))

A [[ 12 -51   4]
 [  6 167 -68]
 [ -4  24 -41]]
Q [[ 0.85714286  0.39428571 -0.33142857]
 [ 0.42857143 -0.90285714  0.03428571]
 [-0.28571429 -0.17142857 -0.94285714]]
R [[  14   21  -14]
 [   0 -175   70]
 [   0    0   35]]
QR [[ 12. -51.   4.]
 [  6. 167. -68.]
 [ -4.  24. -41.]]


A [[ 2. -2. 18.]
 [ 2.  1.  0.]
 [ 1.  2.  0.]]
Q [[ 0.66666667  0.66666667 -0.33333333]
 [ 0.66666667 -0.33333333  0.66666667]
 [ 0.33333333 -0.66666667 -0.66666667]]
R [[ 3.00000000e+00  4.44089210e-16  1.20000000e+01]
 [-4.44089210e-16 -3.00000000e+00  1.20000000e+01]
 [-2.22044605e-16  4.44089210e-16 -6.00000000e+00]]
QR [[ 2.00000000e+00 -2.00000000e+00  1.80000000e+01]
 [ 2.00000000e+00  1.00000000e+00 -4.73695157e-15]
 [ 1.00000000e+00  2.00000000e+00 -2.59052039e-15]]


# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 

# **APPENDIX**

In this appendix some examples are given to express and visualizing mathematical concepts such as vectors, matrices, meshes and functions. 

This is not part of the template report for the course.

# **Vector**

In this cell some basic commands are introduced for how to define and process a vector, where the NumPy array data type is used. 

In [0]:
# Just a test
# Create a vector as a NumPy array
x = np.array([1,2,3,4])

# Print various attributes of the vector
print('x =',x)
print('x[2] =',x[2])
print('x[0:2] =',x[0:2])
print('x.shape =',x.shape)
print('x.ndim =',x.ndim)
print('x.size =',x.size)
print('x.dtype =',x.dtype)

# Print the elements of vector x
for i in range(x.size):
  print("x[",i,"] =",x[i])

# Assign a float value to the int vector x
x[1] = 2.1
print('x =',x)

# Copy the int vector x to a float vector xf
xf = x.astype(float)
xf[1] = 2.1
print('xf =',xf)
print('xf.dtype =',xf.dtype)

# Create a complex vector 
y = np.array([2,3,4], dtype=complex)
print('y =',y)
print('y.dtype =',y.dtype)

# Create a vector z by defining a range through an increment
z = np.arange(12, 28, 4.5)
print('z =',z)

# Overwrite the vector z by a vector of uniformly spaced elements in a range
pi=np.pi
pi
z = np.linspace(0, 2*pi, 5)
print('z =',z)

# Visualize the vector in a plot. 
plt.figure()
plt.plot(x,2*x,'ro-')
plt.show()

# **Matrix**

In this cell some basic commands for defining and processing a matrix
is introduced, where we use the NumPy array data type. 

In [0]:
# Define a matrix as a NumPy array
A = np.array([[2,4,6], [3,6,9], [1,2,1]])

# Print various attributes of the matrix 
print('A =',A)
print('A[1,0] =',A[1,0])
print('A.shape =',A.shape)
print('A.ndim =',A.ndim)
print('A.size =',A.size)

# Plot the matrix
plt.figure()
plt.plot(A,'o-')

# Construct a simple sparse matrix class using the CRS data structure
class spMatrix:
  def __init__(self, val, col_idx, row_ptr):
    self.val = val
    self.col_idx = col_idx
    self.row_ptr = row_ptr
    
# Create a sparse matrix object 
val = np.array([3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3])
col_idx = np.array([1, 2, 4, 2, 3, 3, 3, 4, 5, 5, 6])
row_ptr = np.array([1, 4, 6, 7, 9, 10])
spA = spMatrix(val, col_idx, row_ptr)

# Print sparse matrix object attributes
print('Sparse matrix values: \n',spA.val)
print('Sparse matrix column indices: \n',spA.col_idx)
print('Sparse matrix row pointer: \n',spA.row_ptr)
    

# **Function**

In this cell we illustrate how functions are defined. 

In [0]:
# Define a function f(x,y)
def f(x,y):
  return x + y 

# Verify the function output
print('f(3,4) =',f(3,4))

# Define a function of a function
def g1(fcn,arg1,arg2):
  z = fcn(arg1,arg2)
  print('z =',z)

# Call the function
g1(f,6,7)

# Define a function of a function with arguments in a list or array
def g2(fcn,arg):
  z = fcn(*arg)
  print('z =',z)
  
# Call the function with arguments as list and array
xlist = [8,9]
xarray = np.array([11,3])
g2(f,xlist)
g2(f,xarray)

# Define a function to format the print out for a matrix 
def printMatrix(A):
   rows = A.shape[0]
   cols = A.shape[1]
   for i in range(0,rows):
      for j in range(0,cols):
         print(A[i,j]," ", end='')
      print()

# Define matrix      
A = np.array([[1,2,3], [4,5,6], [7,8,9]])

# Verify the function output
print('A =', A)
print('A =')
printMatrix(A)
print('A^T =')
printMatrix(A.transpose())

# Plot a Euclidian position vector v 
def plotEVector(vec):
  plt.figure()
  plt.xlim([-1.5*vec[0], 1.5*vec[0]])
  plt.ylim([-1.5*vec[1], 1.5*vec[1]])
  plt.quiver(0, 0, vec[0], vec[1], scale=1, scale_units='xy', angles='xy')
  plt.show()

v = np.array([3,4])
plotEVector(v)




# **Plot**

In [0]:
# Define a range for a parameter t
t = np.arange( 0.0, 10, 0.5)

# Initialize a new figure and set the size 
plt.figure(2)
plt.figure(figsize=(12,6))

# Define a subplot structure and choose the first subplot 
plt.subplot(121)

# Plot a standard NumPy function against the parameter t
plt.plot(t,np.cos(t))

# Set plot labels and title for the subplot
plt.xlabel('x')
plt.ylabel('y')
plt.title('My plot: $y = cos(x)$')

# Choose the other subplot
plt.subplot(122)

# Plot three functions in the same subplot
plt.plot(t, np.sin(f(t,3)), 'g*', t, 0.03*t**2, 'ro', t, 0.1*t, 'b')
plt.axis([-1, 12, -2 ,3])

# Show the figure
plt.show()

# Triangular **mesh**

This cell describes how to generate a triangular mesh from a point cloud by the Delaunay algorithm. It also illustrates how to access the mesh data structures. 

In [0]:
# Start the clock 
start = time.time()

# Generate a point cloud
Nx = 5
Ny = 5
hx = 1.0/(Nx-1)
hy = 1.0/(Ny-1)
x = np.zeros((Nx*Ny))
y = np.zeros((Nx*Ny))
for i in range(Nx):
  for j in range(Ny):
    x[i*Nx + j] = j*hx
    y[i*Nx + j] = i*hy

# Generate a mesh from the point cloud using a Delaunay algorithm
mesh = tri.Triangulation( x, y )

# Stop the clock
end = time.time()

# Print the measured time
print ("Mesh generation took: ", end - start, "seconds (wall-clock time)")

# Print mesh data structure
vertices = np.array([mesh.x, mesh.y])
print('vertices = \n', vertices)
print('edges = \n', mesh.edges)
print('triangles = \n', mesh.triangles)
print('neighbor triangles = \n', mesh.neighbors)

# Plot the mesh 
plt.figure(3)
plt.figure(figsize=(10,10))
plt.triplot(mesh, 'bo-', lw=1)
plt.title('Delaunay mesh')
plt.show()

# Define a function g(x,y)
def g(x,y):
    return np.sin(10*x)*np.cos(3*y)

# Plot the function g(x,y) in a contour plot
plt.figure(4)
plt.tricontour(mesh, g(x,y), cmap=plt.cm.spectral)
plt.show()

# Plot the function g(x,y) in 3D as a surface plot
fig = plt.figure(5)
ax1 = fig.add_subplot(111, projection='3d')
surf = ax1.plot_trisurf(mesh, g(x,y), cmap=plt.cm.coolwarm)
fig.colorbar(surf, shrink=0.5, aspect=10)
ax1.view_init(20,10)
plt.show()

# Parametric **curve**

This cell illustrates how to visualize a parametric curve.

In [0]:
# Define paramter range
t = np.linspace(0, 4*np.pi, 100)

# Define the curve
x = t*np.cos(t)
y = t*np.sin(t)
z = t

fig = plt.figure(6)
ax = fig.gca(projection = '3d')
ax.plot(x, y, z)
plt.title('Parametric curve')
plt.show()